In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### You can find the dataset from [here](https://www.kaggle.com/datasets/sp1thas/book-depository-dataset)

In [2]:
authors=pd.read_csv('../input/book-depository-dataset/authors.csv')
categories=pd.read_csv('../input/book-depository-dataset/categories.csv')
dataset=pd.read_csv('../input/book-depository-dataset/dataset.csv')
formats=pd.read_csv('../input/book-depository-dataset/formats.csv')
places=pd.read_csv('../input/book-depository-dataset/places.csv')

In [3]:
formats.head()

In [4]:
dataset.head()

In [5]:
dataset.info()

In [76]:
dataset.isbn10.value_counts()

In [78]:
popular_df=pd.read_csv('../input/popular-dfcsv/popular_df.csv')

In [81]:
popular_df.shape

In [82]:
books=pd.read_csv('../input/bookscsv/Books.csv')
print(books.shape)
books=books[books['Book-Title'].isin(popular_df['Book-Title'].values)]
print(books.shape)
books.head()

In [93]:
dataset[dataset.isbn10.isin(books['ISBN'].values)].title.reset_index().title.value_counts()

In [84]:
books['Book-Title'].value_counts()

In [ ]:
dataset[dataset.isbn10.isin(books)]

In [6]:
new_dataset=dataset.drop(['dimension-x','dimension-y','dimension-z','edition','edition-statement','isbn10','isbn13','publication-date','publication-place','imprint','weight','bestsellers-rank','illustrations-note','index-date','for-ages'],axis=1)

In [7]:
new_dataset.info()

In [8]:
new_dataset.isnull().sum()

In [9]:
new_dataset.shape

In [10]:
new_dataset.lang.value_counts()

In [11]:
new_dataset=new_dataset[new_dataset.lang=='en']

In [12]:
new_dataset.isnull().sum()

In [13]:
new_dataset[new_dataset.title=='1984']

In [14]:
formats.info()

In [15]:
formats.format_name.values

In [16]:
formats.rename({'format_id':'format'},axis=1,inplace=True)

In [17]:
new_dataset=new_dataset.merge(formats)

In [18]:
formats.format.values

In [19]:
new_dataset=new_dataset[new_dataset.format_name=='Book']

In [20]:
new_dataset.shape

In [21]:
len(new_dataset.title.unique())

In [22]:
temp=new_dataset.title.value_counts().reset_index()
temp[temp.title>1].head()

In [23]:
new_dataset[new_dataset.title=='Dinosaurs']

In [24]:
new_dataset.drop(['format','format_name'],axis=1,inplace=True)

In [25]:
new_dataset.info()

In [26]:
authors.info()

In [27]:
authors.head()

In [28]:
authors.isnull().sum()

In [29]:
authors[authors.author_id==520].author_name.values[0]

In [30]:
import json
def add_author(L):
    auth=[]
    L = json.loads(L)
    for i in L:
        auth_name = authors[authors['author_id']==i].author_name.values[0]
        auth.append(auth_name)
    return auth

In [31]:
new_dataset.authors=new_dataset.authors.apply(add_author)

In [32]:
categories.info()

In [33]:
def add_category(L):
    cat=[]
    L = json.loads(L)
    for i in L:
        cat_name = categories[categories['category_id']==i].category_name.values[0]
        cat.append(cat_name)
    return cat

In [34]:
new_dataset.categories=new_dataset.categories.apply(add_category)

In [35]:
new_dataset.head()

In [36]:
new_dataset[['image-checksum','image-path','image-url']].head(1).values

In [37]:
popular_df=pd.read_csv('../input/popular-dfcsv/popular_df.csv')
popular_df.head()

In [38]:
titles=popular_df['Book-Title'].values
new_dataset[new_dataset.title.isin(titles)].shape

In [39]:
new_dataset.drop(['image-checksum','image-path','lang','rating-avg','rating-count'],axis=1,inplace=True)

In [40]:
new_dataset.head()

In [41]:
new_dataset.info()

In [42]:
new_dataset.isnull().sum()

In [43]:
temp=new_dataset.title.value_counts().reset_index()
temp[temp['title']>1]['index']

Removing the rows having `nan` data in `description` and duplicate `title`

In [44]:
t=new_dataset[new_dataset.title.isin(temp[temp['title']>1]['index'])]
new_dataset.drop(new_dataset[new_dataset.id.isin(t[t.description.isnull()].id)].index,inplace=True)


In [45]:
new_dataset.drop_duplicates('title',inplace=True)

In [46]:
new_dataset.isnull().sum()

In [47]:
new_dataset.info()

In [48]:
new_dataset.head()

In [49]:
new_dataset=new_dataset.reset_index().drop(['index'],axis=1)

In [50]:
new_dataset.head()

In [51]:
new_dataset.fillna('', inplace=True)

In [52]:
len(list(map(lambda x:x.split(),new_dataset.description.values)))

In [53]:
new_dataset.info()

In [54]:
import re
new_dataset.description=new_dataset.description.apply(lambda x: re.sub(r'[^\w]', ' ', x))

In [55]:
new_dataset['desc1'] = new_dataset.description.apply(lambda x:x.split())

In [56]:
new_dataset['tags']=new_dataset.authors+new_dataset.categories+new_dataset.desc1

In [57]:
type(new_dataset.tags)

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [59]:
new=new_dataset[['title','tags']]

In [60]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

In [61]:
import nltk

In [62]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [63]:
def stem(txt):
    y=[]
    for text in txt.split():
        y.append(ps.stem(text))
    
    return ' '.join(y)

In [64]:
new['tags']=new.tags.apply(stem)

In [65]:
vector = cv.fit_transform(new['tags']).toarray()

In [66]:
vector.shape

In [67]:
from sklearn.metrics.pairwise import cosine_similarity
similar=cosine_similarity(vector)

In [68]:
similar.shape

In [69]:
new_dataset.info()

In [70]:
def suggest(book):
    index=new[new.title==book].index[0]
    recommend=similar[index]
    books=sorted(enumerate(recommend),key=lambda x: x[1],reverse=True)[1:9]
    book_details=[]
    for bk in books:
        bk_index=bk[0]
        book_details.append(new_dataset[['authors','image-url','title','url','description']].values.tolist()[bk_index])
    return book_details

In [71]:
new[new.title=='Diet and Fitness Journal'].index[0]

In [72]:
suggest('Diet and Fitness Journal')

In [73]:
import pickle

In [74]:
pickle.dump(new,open('new.pkl','wb'))
pickle.dump(new_dataset,open('new_dataset.pkl','wb'))
pickle.dump(similar,open('similar.pkl','wb'))
